<a href="https://colab.research.google.com/github/Chirag314/Time-series-forecasting/blob/main/Copy_of_Untitled9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import urllib
import zipfile

import pandas as pd
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras
import numpy as np

# This function downloads and extracts the dataset to the directory that
# contains this file.
# DO NOT CHANGE THIS CODE
# (unless you need to change https to http)
def download_and_extract_data():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/household_power.zip'
    urllib.request.urlretrieve(url, 'household_power.zip')
    with zipfile.ZipFile('household_power.zip', 'r') as zip_ref:
        zip_ref.extractall()


# This function normalizes the dataset using min max scaling.
# DO NOT CHANGE THIS CODE
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data


# This function is used to map the time series dataset into windows of
# features and respective targets, to prepare it for training and
# validation. First element of the first window will be the first element of
# the dataset. Consecutive windows are constructed by shifting
# the starting position of the first window forward, one at a time (indicated
# by shift=1). For a window of n_past number of observations of all the time
# indexed variables in the dataset, the target for the window
# is the next n_future number of observations of these variables, after the
# end of the window.

# DO NOT CHANGE THIS CODE
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)




In [ ]:
download_and_extract_data()
    # Reads the dataset from the CSV.
df = pd.read_csv('household_power_consumption.csv', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)

In [ ]:
len(df)

86400

In [ ]:
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [ ]:
N_FEATURES = len(df.columns) # DO NOT CHANGE THIS

    # Normalizes the data
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

    # Splits the data into training and validation sets.
SPLIT_TIME = int(len(data) * 0.5) # DO NOT CHANGE THIS
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

    # DO NOT CHANGE THIS CODE
tf.keras.backend.clear_session()
tf.random.set_seed(42)

    # DO NOT CHANGE BATCH_SIZE IF YOU ARE USING STATEFUL LSTM/RNN/GRU.
    # THE TEST WILL FAIL TO GRADE YOUR SCORE IN SUCH CASES.
    # In other cases, it is advised not to change the batch size since it
    # might affect your final scores. While setting it to a lower size
    # might not do any harm, higher sizes might affect your scores.
BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS

    # DO NOT CHANGE N_PAST, N_FUTURE, SHIFT. The tests will fail to run
    # on the server.
    # Number of past time steps based on which future observations should be
    # predicted
N_PAST = 24  # DO NOT CHANGE THIS

    # Number of future time steps which are to be predicted.
N_FUTURE = 24  # DO NOT CHANGE THIS

    # By how many positions the window slides to create a new window
    # of observations.
SHIFT = 1  # DO NOT CHANGE THIS

    # Code to create windowed train and validation datasets.
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)


In [ ]:
data.shape

(86400, 7)

In [ ]:
print(f"Shape of training data: {x_valid.shape}")


Shape of training data: (43200, 7)


In [ ]:
for element in train_set:
  print(element)
  import sys
  sys.exit()

(<tf.Tensor: shape=(32, 24, 7), dtype=float64, numpy=
array([[[0.43377912, 0.47826087, 0.04036551, ..., 0.        ,
         0.01282051, 0.85      ],
        [0.55716135, 0.49885584, 0.0355582 , ..., 0.        ,
         0.01282051, 0.8       ],
        [0.55867127, 0.56979405, 0.03420739, ..., 0.        ,
         0.02564103, 0.85      ],
        ...,
        [0.81018119, 0.        , 0.0250298 , ..., 0.        ,
         0.        , 0.85      ],
        [0.73684211, 0.        , 0.02991657, ..., 0.        ,
         0.        , 0.8       ],
        [0.53710095, 0.        , 0.03778308, ..., 0.        ,
         0.        , 0.85      ]],

       [[0.55716135, 0.49885584, 0.0355582 , ..., 0.        ,
         0.01282051, 0.8       ],
        [0.55867127, 0.56979405, 0.03420739, ..., 0.        ,
         0.02564103, 0.85      ],
        [0.56018119, 0.57437071, 0.03599523, ..., 0.        ,
         0.01282051, 0.85      ],
        ...,
        [0.73684211, 0.        , 0.02991657, ..., 0.  

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
x_train.shape

(43200, 7)

In [ ]:
model = tf.keras.models.Sequential([

        # ADD YOUR LAYERS HERE.

        tf.keras.layers.Conv1D(filters=32, kernel_size=5,
                               strides=1, padding="causal",
                               activation="relu",
                               input_shape=[24, 7]),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),

        # If you don't follow the instructions in the following comments,
        # tests will fail to grade your code:
        # The input layer of your model must have an input shape of:
        # (BATCH_SIZE, N_PAST = 24, N_FEATURES = 7)
        # The model must have an output shape of:
        # (BATCH_SIZE, N_FUTURE = 24, N_FEATURES = 7).
        # Make sure that there are N_FEATURES = 7 neurons in the final dense
        # layer since the model predicts 7 features.

        # HINT: Bidirectional LSTMs may help boost your score. This is only a
        # suggestion.

        # WARNING: If you are using the GRU layer, it is advised not to use the
        # recurrent_dropout argument (you can alternatively set it to 0),
        # since it has not been implemented in the cuDNN kernel and may
        # result in much longer training times.
        tf.keras.layers.Dense(N_FEATURES)
        
    ])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 24, 32)            1152      
                                                                 
 lstm (LSTM)                 (None, 24, 64)            24832     
                                                                 
 lstm_1 (LSTM)               (None, 24, 64)            33024     
                                                                 
 dense (Dense)               (None, 24, 30)            1950      
                                                                 
 dense_1 (Dense)             (None, 24, 10)            310       
                                                                 
 dense_2 (Dense)             (None, 24, 7)             77        
                                                                 
Total params: 61,345
Trainable params: 61,345
Non-traina

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.compile(
        # YOUR CODE HERE
         loss="mse", optimizer = tf.keras.optimizers.SGD(learning_rate=1e-5),metrics=[tf.metrics.MeanAbsoluteError()]
    )

In [ ]:
model.fit(
        # YOUR CODE HERE
        train_set, epochs=5,validation_data=valid_set, callbacks=[ early_stopping], verbose=1
    )

Epoch 1/5
1349/1349 [==============================] - 58s 41ms/step - loss: 0.0644 - mean_absolute_error: 0.1367 - val_loss: 0.0647 - val_mean_absolute_error: 0.1335
Epoch 2/5
1349/1349 [==============================] - 46s 34ms/step - loss: 0.0639 - mean_absolute_error: 0.1366 - val_loss: 0.0642 - val_mean_absolute_error: 0.1334
Epoch 3/5
1349/1349 [==============================] - 46s 34ms/step - loss: 0.0634 - mean_absolute_error: 0.1366 - val_loss: 0.0637 - val_mean_absolute_error: 0.1333
Epoch 4/5
1349/1349 [==============================] - 46s 34ms/step - loss: 0.0629 - mean_absolute_error: 0.1365 - val_loss: 0.0633 - val_mean_absolute_error: 0.1332
Epoch 5/5
1349/1349 [==============================] - 46s 34ms/step - loss: 0.0625 - mean_absolute_error: 0.1364 - val_loss: 0.0628 - val_mean_absolute_error: 0.1331
